# MBA DSA USP/ESALQ 2024.1 Time 2
## Desafio House Prices

### Limpeza e ajuste dos dados

Importação das bibliotecas necessárias

In [102]:
import pandas as pd

Leitura dos dados do arquivo para treino ("train.csv") e armazenando no objeto 'dados_brutos'

In [ ]:
dados_brutos = pd.read_csv("train.csv")

print("Quantidade de linhas e colunas")
print(dados_brutos.shape)

print("\nTipos de objetos do DataFrame")
print(dados_brutos.dtypes)

print("\nInformações sobre o banco de dados (coluna, contador de nulos, e tipo de informação)")
dados_brutos.info()

O banco de dados possui uma série de colunas (variáveis) que possuem valores nulos, algumas em maior quantidade e outras em menor quantidade:

In [ ]:
nulos = dados_brutos.isnull().sum()
nulos_ordenados = nulos.sort_values(ascending=False)

print("Quantidade de valores nulos por coluna:")
print(nulos_ordenados.head(20))
print("Como o último valor já é zero, depois desse todos são zerados mesmo")


Como são valores absolutos, é mais fácil excluir do banco de dados as variáveis que possuam valores nulos acima de um determinado percentual do total (eu usei 15% como delimitador), salvando no objeto 'dados_filtrados'

In [ ]:
nulos_percentuais = nulos/len(dados_brutos.index)
var_excluir = dados_brutos.columns[nulos_percentuais > 0.15]
dados_filtrados = dados_brutos.drop(var_excluir, axis = 1)

print("Quantidade de linhas e colunas depois de ajustado")
print(dados_filtrados.shape)

print("\nTipos de objetos do DataFrame")
print(dados_filtrados.dtypes)

print("\nInformações sobre o banco de dados (coluna, contador de nulos, e tipo de informação)")
dados_filtrados.info()

Isso feito, existem variáveis que são 'object' (não-numérico, então qualitativas ou categóricas) e variáveis que são "intXX" (portanto, numéricas). Por questão de facilidade e considerando o tempo disponível, por enquanto vou desconsiderar as variáveis 'quali' e vou utilizar apenas as 'quanti'.

In [ ]:
colunas_quali = dados_filtrados.columns[dados_filtrados.dtypes == "object"]
variaveis_quanti = dados_filtrados.drop(colunas_quali, axis = 1)
print("AS VARIÁVEIS NUMÉRICAS ('QUANTI') ENTÃO SÃO 37:")
print(variaveis_quanti.columns)
print("\n")

colunas_quanti = dados_filtrados.columns[dados_filtrados.dtypes != "object"]
variaveis_quali = dados_filtrados.drop(colunas_quanti, axis = 1)
print("AS VARIÁVEIS NÃO-NÚMERICAS ('QUALI') FICAM SENDO APENAS 38:")
print(variaveis_quali.columns)

 Filtro das variáveis quali e quanti para evitar confusão. Contudo, ainda existem valores nulos no meio dos dados (foram eliminados só as variações com nulos >= 15% do total). Necessário realizar a imputação esses valores, pois nenhuma análise é feita com qualquer valor nulo: quebra o sistema.

 Link sobre o carregamento das bibliotecas para imputar dados: https://medium.com/@sanjushusanth/missing-value-imputation-techniques-in-python-62aeab65a6a6

 Por mera facilidade momentânea, serão utilizados os dados de média e mediana para as imputações, ou, ainda seguindo o que alguns consideram 'boas práticas', simplesmente registrando um '-1' no valor imputado. Fiz um banco de dados para cada tipo dessas imputações a fim de testar qual melhor se encaixa nessa simples imputação (sem cálculos próprios de imputação para esse fim).

In [ ]:
variaveis_quanti_namum = variaveis_quanti.fillna(-1)
variaveis_quanti_namediana = variaveis_quanti.fillna(variaveis_quanti.median())
variaveis_quanti_namedia = variaveis_quanti.fillna(variaveis_quanti.mean())

print("VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO '-1' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namum.isnull().sum() > 0])

print("\n VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MÉDIA' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namedia.isnull().sum() > 0])

print("\n VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MEDIANA' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namediana.isnull().sum() > 0])

try:
    variaveis_quanti_namum.columns[variaveis_quanti_namediana.isnull().sum() > 0] + variaveis_quanti_namum.columns[variaveis_quanti_namedia.isnull().sum() > 0]+ variaveis_quanti_namum.columns[variaveis_quanti_namum.isnull().sum() > 0]
    print("\nTUDO CERTO!")
except:
    print("\n!!! DEU ALGUM ERRO !!!")

Definição da variável dependente (y) e das variáveis independentes (x) de cada um dos bancos de dados anteriores:

In [95]:
x_namum = variaveis_quanti_namum.drop("SalePrice", axis = 1)
x_namedia = variaveis_quanti_namedia.drop("SalePrice", axis = 1)
x_namediana = variaveis_quanti_namediana.drop("SalePrice", axis = 1)
y = variaveis_quanti.SalePrice # como o 'y' é igual para todos (o preço de venda), então tanto faz a base de dados escolhida

### Separação dos dados para treino e teste

Importação das bibliotecas adicionais necessárias

In [105]:
from sklearn.model_selection import train_test_split

Separação do banco de dados entre treino (0,80 ou 80%) e teste (0,20 ou 20%) - proporções arbitrárias, eu que defini assim. Pode-se testar com proporções diferentes. Usa-se o scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Para garantir consistência entre os resultados, utilizado sempre _random_state = 1_

Para os dados imputados com -1:

In [ ]:
treino_namum, teste_namum, treino_y, teste_y = train_test_split(x_namum, y, test_size = 0.2, random_state = 1)

print("DADOS TREINO NA COM -1")
print(treino_namum.head(5))

print("\nDADOS TESTE NA COM -1")
print(teste_namum.head(5))

print("\n DADOS TREINO SalePrice")
print(treino_y.head(5))

print("\nDADOS TESTE SalePrice")
print(teste_y.head(5))

Para os dados imputados com a média:

In [ ]:
treino_namedia, teste_namedia, treino_y, teste_y = train_test_split(x_namedia, y, test_size = 0.2, random_state = 1)

print("DADOS TREINO NA COM MÉDIA")
print(treino_namedia.head(5))

print("\nDADOS TESTE NA COM MÉDIA")
print(teste_namedia.head(5))

print("\n DADOS TREINO SalePrice")
print(treino_y.head(5))

print("\nDADOS TESTE SalePrice")
print(teste_y.head(5))

Para os dados imputados com a mediana:

In [ ]:
treino_namediana, teste_namediana, treino_y, teste_y = train_test_split(x_namediana, y, test_size = 0.2, random_state = 1)

print("DADOS TREINO NA COM MEDIANA")
print(treino_namediana.head(5))

print("\nDADOS TESTE NA COM MEDIANA")
print(teste_namediana.head(5))

print("\n DADOS TREINO SalePrice")
print(treino_y.head(5))

print("\nDADOS TESTE SalePrice")
print(teste_y.head(5))

### Modelagem
Com os dados já limpos, já separados pelas imputações diferentes para verificação e com as bases para treino e teste, começa-se efetivamente a modelagem através de alguns algoritmos diferentes a fim de testar precisão dos resultados estimados.

Importação das bibliotecas adicionais necessárias

In [109]:
from sklearn.linear_model import LinearRegression # biblioteca para modelagem através de regressão Linear Simples
from sklearn import tree # biblioteca para modelagem através de árvore de decisão
from sklearn.neighbors import KNeighborsRegressor # biblioteca para modelagem através de KNN

#### NaN imputado com -1

##### Regressão Linear Simples
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

In [ ]:
treino_namum, teste_namum, treino_y, teste_y = train_test_split(x_namum, y, test_size = 0.2, random_state = 1)

namum_rl = LinearRegression().fit(treino_namum, treino_y)
y_namum_rl = namum_rl.predict(teste_namum)

##### Árvore de Decisão
https://scikit-learn.org/stable/modules/tree.html#regression

In [ ]:
namum_tree = tree.DecisionTreeRegressor(randomm_state = 1).fit(treino_namum, treino_y)
y_namum_tree = namum_tree.predict(teste_namum)

##### K Vizinhos Mais Próximos (KNN - K Nearest Neighbors)
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

In [ ]:
namum_knn = KNeighborsRegressor(n_neighbors = 2).fit(treino_namum, treino_y)
y_namum_knn = namum_knn.predict(teste_namum)

### Avaliação dos resultados
Realizada através de RMSE (root mean squared error), no caso deste desafio.

Importação das bibliotecas adicionais necessárias

In [110]:
from sklearn.metrics import root_mean_squared_error

#### Análises (pela imputação)

##### NaN por -1